<h2><center><b><i>Cluster bomb</b></i>: Uncovering Patterns in Terrorist Group Beliefs and Attacks</center></h2>

#### **COM-480: Data Visualization**

**Team**: Alexander Sternfeld, Silvia Romanato & Antoine Bonnet

**Dataset**: [Global Terrorism Database (GTD)](https://www.start.umd.edu/gtd/) 

**Additional dataset**: [Profiles of Perpetrators of Terrorism in the United States (PPTUS)](https://dataverse.harvard.edu/dataset.xhtml?persistentId=hdl%3A1902.1/17702)

## **Cluster analysis**
 

In [6]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from load_data import *

pd.set_option('display.max_columns', None)

GTD = load_GTD()
PPTUS_data, PPTUS_sources = load_PPTUS()

GTD pickle file found, loading...
PPTUS pickle files found, loading...


In [7]:
# Find all terrorist attacks with gname non null
known_groups = GTD[GTD['gname'] != 'Unknown']
print(known_groups.shape)
known_groups.gname.value_counts().head(30)

(120991, 135)


Taliban                                             12936
Islamic State of Iraq and the Levant (ISIL)          7479
Shining Path (SL)                                    4567
Al-Shabaab                                           4547
Houthi extremists (Ansar Allah)                      3516
Boko Haram                                           3459
New People's Army (NPA)                              3441
Farabundo Marti National Liberation Front (FMLN)     3351
Irish Republican Army (IRA)                          2670
Kurdistan Workers' Party (PKK)                       2612
Revolutionary Armed Forces of Colombia (FARC)        2490
Maoists                                              2164
Communist Party of India - Maoist (CPI-Maoist)       2113
Basque Fatherland and Freedom (ETA)                  2024
National Liberation Army of Colombia (ELN)           1842
Liberation Tigers of Tamil Eelam (LTTE)              1602
Tehrik-i-Taliban Pakistan (TTP)                      1509
Fulani extremi

In [9]:
terrorist_groups = list(PPTUS_data.ORGNAME)
terrorist.groups.append(list(PPTUS_data.))
print(terrorist_groups)

['Action Squad', 'Al-Qaida', 'Al-Qaida in the Arabian Peninsula (AQAP)', 'American Indian Movement', "American Servicemen's Union (ASU)", 'Americans for a Competent Federal Judicial System', 'Americans for Justice', 'Animal Liberation Front (ALF)', 'Anti-Castro Command', 'Antonia Martinez Student Commandos (AMSC)', 'Armed Commandos of Liberation', 'Armed Commandos of Student Self Defense', 'Armed Forces of Popular Resistance (FARP)', 'Armed Revolutionary Independence Movement (MIRA)', 'Armenian Secret Army for the Liberation of Armenia', 'Army of God', 'Aryan Liberation Front', 'Aryan Nation', 'Black Afro Militant Movement', 'Black Brigade (United States)', 'Black Liberation Army', 'Black Panthers', 'Black Revolutionary Assault Team', 'Black September', 'Boricua Revolutionary Front', 'Boricuan Armed Anti-Imperialist Commandos', 'Chicano Liberation Front', 'Citizens for Constitutional Freedom', 'Comrades in Arms', 'Continental Revolutionary Army', 'Coordination of the United Revolutiona

In [11]:
# Find all terrorist attacks in GTD whose gname is in either PPTUS.ORGNAME or PPTUS.ALIAS

GTD[GTD.gname in terrorist_groups]


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [11]:
PPTUS_data.ALIAS

0                                                    NaN
1      al-Qaida; al-Qaida; The Base; Qaida al-Jihad; ...
2      Al Qaida of Jihad Organization in the Arabian ...
3                                                    AIM
4                                                    ASU
                             ...                        
140    Weathermen; Weather Underground Organization; ...
141                                                  NaN
142                                           Joven Cuba
143                                                  YOS
144                        Death Angels; Zebra Murderers
Name: ALIAS, Length: 145, dtype: object